In [185]:
""" Default stuff for Dn quiver. """

import sympy as sp
from queue import Queue
from collections import defaultdict


class Quiver:
    def __init__(self, Q_0: list, Q_1: list[tuple], 
                 tau: list[tuple]=None) -> None:
        self.vertices = []
        for v in Q_0:
            if isinstance(v, tuple):
                v = f"{v[0]}{''.join(str(x) for x in v[1].vec())}"
            self.vertices.append(v)
        self.vertices.sort(key=lambda x: x[0] if isinstance(x, tuple) else x)
        self.arrows = defaultdict(list)
        for a, b in Q_1:
            if isinstance(a, tuple):
                a = f"{a[0]}{''.join(str(x) for x in a[1].vec())}"
            if isinstance(b, tuple):
                b = f"{b[0]}{''.join(str(x) for x in b[1].vec())}"
            self.arrows[a].append(b)
        for a in self.arrows:
            self.arrows[a].sort()
        self.translations = tau

    def __repr__(self) -> str:
        repr = "-" * 20 + "Q_0" + "-" * 20 + "\n"
        repr += ', '.join(str(x) for x in self.vertices) + "\n\n"
        repr += "-" * 20 + "Q_1" + "-" * 20 + "\n"
        for a in self.arrows:
            repr += f"{a} -> " + ', '.join(str(a) for a in self.arrows[a]) +\
                "\n"

        return repr
    
    def __getitem__(self, key: int) -> list[int]:
        return self.arrows.get(key, [])
    
    def __iter__(self):
        for v in self.vertices:
            yield v

    def dual(self):
        Q_0 = [v for v in self]
        Q_1 = []
        for a in self.arrows:
            for b in self.arrows[a]:
                Q_1.append((b, a))
        return Quiver(Q_0, Q_1)

m = 2
# canonical basis
e = [None] + [
    sp.Matrix([0] * (i - 1) + [1] + [0] * (m - i)) for i in range(1, m + 1)
]
def ar_translate(M: sp.Matrix) -> sp.Matrix:
    """ Auslander-Reiten translate of a module. """
    return A * M

def l_simplex(l: int, n: int) -> set:
    """ l-simplex of dimension n for an integer l. """
    D = []
    q = Queue()
    q.put(sp.Matrix([0] * n))
    while not q.empty():
        L = q.get()
        if sum(L) <= l:
            D.append(L)
            for i in range(1, n + 1):
                L += e[i]
                q.put(L.copy())
                L -= e[i]

    return D

# D_n quiver
n = 4
Q_0 = list(range(1, n + 1))
Q_1 = [(i, i - 1) for i in range(5, n + 1)] + [(i, 1) for i in range(2, 5)]
Q = [Quiver(Q_0, Q_1)]

# Indecomposable projectives
P = [None] * (n + 1)
for i in range(1, n):
    P[i] = sp.Matrix([1] * i + [0] * (n - i))
P[n] = sp.Matrix([1] * (n - 2) + [0, 1])

# Cartan matrix
C = sp.Matrix(
    [P[i].T for i in range(1, len(P))]
).T
# Indecomposable injectives
I = [None] + [
    C.row(i-1).T for i in range(1, len(P))
]
# Coxeter matrix
A = -C.T * C**-1

# Maximum number of translations of injectives
l = [None] + [0] * (len(I) - 1)
for i in range(1, len(I)):
    M = I[i]
    while M not in P:
        M = ar_translate(M)
        l[i] += 1

In [187]:
""" Calculating D_n^(m) quiver. """

# l-simplices for indecomposable injectives
D = [None] + [
    l_simplex(l[i], m) for i in range(1, n + 1)
]
# some other basis
v = [None] * (m + 1)
for i in range(1, m + 1):
    if i == 1:
        v[i] = -e[i]
    else:
        v[i] = e[i - 1] - e[i]
# vertices
Q_0 = [
    (x, l) for x in Q[0] for l in D[x]
]
# arrows
Q_1 = [
    ((x, l), (w, l)) for x in Q[0] for w in Q[0] 
    for l in D[x] 
    if x in Q[0][w]
] + [
    ((x, l), (y, l + v[1])) 
    for x in Q[0] for y in Q[0] 
    for l in D[x] 
    if y in Q[0][x] and (y, l + v[1]) in Q_0
] + [
     ((x, l), (x, l + v[i])) for x in Q[0] for i in range (2, m + 1)
]
Q.append(Quiver(Q_0, Q_1))
Q[1].dual()

TypeError: cannot add <class 'sympy.matrices.dense.MutableDenseMatrix'> and <class 'list'>